In [1]:
import os
from langchain_core.prompts.prompt import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_neo4j import Neo4jGraph, Neo4jVector, GraphCypherQAChain
from langchain_experimental.graph_transformers import LLMGraphTransformer

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Load models
llm = ChatOllama(model=os.getenv('OLLAMA_DEFAULT_LLM', 'gemma3:4b'), base_url=os.getenv('OLLAMA_BASE_URL'))
embedding = OllamaEmbeddings(model=os.getenv('OLLAMA_DEFAULT_EMBEDDING', 'nomic-embed-text'), base_url=os.getenv('OLLAMA_BASE_URL'))

# Load Neo4j instance
graph = Neo4jGraph(
    url='bolt://localhost:7687',
    username='neo4j',
    password='none',
    database='neo4j',
    timeout=None,
    refresh_schema=False,
    enhanced_schema=True,
)

# Load LLMGraphTransformer
additional_instructions = """
Make sure to comply with the following requirements:
- For each node, provide a `description` based only on the provided inputs.
- For each edge, use concise relationship types with symmetric relationships having bidirectional edges.
"""

llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[],
    allowed_relationships=[],
    prompt=None,
    node_properties=True,
    relationship_properties=True,
    additional_instructions=additional_instructions,
)

In [ ]:
# Sample text
text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
Marie Curie was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Marie Curie's husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
Marie Curie was, in 1906, the first woman to become a professor at the University of Paris.
"""

# Load and chunk documents
text_splitter = RecursiveCharacterTextSplitter(separators=['. ', '\n'], chunk_size=150, chunk_overlap=20)
documents = text_splitter.create_documents(texts=[text])

# Transform documents to Knowledge Graph
graph_documents = llm_transformer.convert_to_graph_documents(documents=documents)
graph.add_graph_documents(graph_documents=graph_documents, include_source=True, baseEntityLabel=True)
graph.refresh_schema()  # Refresh schema for retrieval below

# Create a vector store for documents
vector_store = Neo4jVector.from_existing_graph(
    graph=graph,
    embedding=embedding,
    node_label='Document',
    embedding_node_property='embedding',
    text_node_properties=['text'],
    # keyword_index_name='keyword',
    index_name='document_index',
    # search_type='hybrid',
)
vector_store.retrieval_query = vector_store.retrieval_query.replace('id: Null, ', '')    # Include node id as metadata

In [ ]:
# Create retriever
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 1})

# Create GraphCypher retriever
cypher_prompt = """
Task: Generate Cypher statement to query a graph database.

Instructions:
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Notes: 
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
- Do not include any text except the generated Cypher statement.

Question:
{question}
"""

graph_retriever = GraphCypherQAChain.from_llm(
    top_k=1,
    llm=llm,
    graph=graph,
    cypher_prompt=PromptTemplate.from_template(cypher_prompt),
    cypher_llm=llm,
    # exclude_types=[],
    # include_types=[],
    validate_cypher=True,
    allow_dangerous_requests=True,
    verbose=True,
    return_direct=True,
    # use_function_response=True,
    # return_intermediate_steps=True,
)

# Sample query
query = 'Who is the husband of Marie Curie?'
source_id = retriever.invoke(query)[0].metadata.get('id')
graph_query = f'Use document {source_id} as context. {query}'
graph_retriever.invoke({'query': graph_query})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document {id: 'ac98d0e62702d7056d7e6b8c8ce22388'})-[:MENTIONS]->(p:Person {id: 'Marie Curie'})-[:MARRIED_TO]->(h:Person)
RETURN h.id AS Husband


> Finished chain.


{'query': 'Use document ac98d0e62702d7056d7e6b8c8ce22388 as context. Who is the husband of Marie Curie?',
 'result': [{'Husband': 'Pierre Curie'}]}